### Need to generate Adult Data and pass it in self.data
<ul>
    <li>Need ask best params from Rosin</li>
    <li> run Adult
    <li> Save data from PreProcessing somewhere
    <li> Wrap to take two csv of corresponding original (label)
    <li> Code test testing loop
    <li> What to use as test loss and final metric
    <li> Same Encoding
</ul>


### Imports

In [9]:
# # Imports
import utilities as utils
# import datasets as d

# from Stats import Plots as Pl
import tqdm
import yaml
import json
import torch
import os.path
import warnings
import importlib
import random
import torchvision
import numpy as np
import pandas as pd
from torch import nn
import seaborn as sns
from typing import List
from torch.utils.data import *
import matplotlib.pyplot as plt
from torch.utils import data as td
from joblib import Parallel, delayed

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
CPU_DEVICE = torch.device("cpu")
GET_VALUE = lambda x: x.to(CPU_DEVICE).data.numpy().reshape(-1)[0]
print(f"\n *** \n Currently running on {DEVICE}\n *** \n")


 *** 
 Currently running on cpu
 *** 



In [8]:
# Data import & Pre-processing
  
class My_dataLoader:
    def __init__(self, batch_size : int, path :str, n_train :int,  label_col_name:str, test_batch_size:int=128):
        '''
            Creates train and test loaders from local files, to be easily used by torch.nn
            
            :batch_size: int for size of training batches
            :path: path to csv where data is. 2d file, where last value of each cols
                    is the label
            :n_train: int for the size of training set (assigned randomly)
            :test_batch: size of batches at test time. If none, will be same 
                    as training
            :label_col_name name of columns that contains labels
        '''
        self.batch_size = batch_size
        self.path = path
        self.train_size = n_train
        self.test_batch = test_batch_size
        
        df = pd.read_csv(path)
        
        
        a = df.values
        self.data = torch.tensor(a[:,:-1]) # where data is 2d [D_train_size x features]
        self.labels = torch.tensor(a[:,-1])
        
        self.local_dataset = torch.utils.data.TensorDataset(self.data, self.labels)
                     
        #custom_transform = transforms.Normalize((mean_mean,), (std_mean,)) 
        
        
        indices = list(range(len(self.local_dataset)))
        random.shuffle(indices)
        
        # Split dataset into train and Test sets
        self.train_loader = torch.utils.data.DataLoader(
            self.local_dataset,
            batch_size=self.batch_size,
            sampler=SubsetRandomSampler(indices[:self.train_size]),
            num_workers=1,
            pin_memory=False
        )

        self.test_loader = DataLoader(
            self.local_dataset,
            batch_size=self.test_batch,
            sampler=SubsetRandomSampler(indices[self.train_size:]),
            num_workers=1,
            pin_memory=False
        )

class PreProcessing:
    def __init__(self, params_file: str):
        '''
            Imports all variables/parameters necessary for preparation of training.
            Looks into params.yaml, then creates dataloader that can be used
            
            params_file: path to file that contains parameters.
                            Needs to follow a specific naming and format
        '''
        # Import params
        stream = open(params_file, 'r')
        data = yaml.load(stream, yaml.FullLoader)

        import_path = data['data_loading']['path']['value']

        batchSize = data['model_params']['batchSize']['value']
        test_batch_size = data['model_params']['test_batch_size']['value']
        percent_train_set = data['model_params']['percent_train_set']['value']


        df2 = pd.read_csv(import_path+".csv")
        n_test = int(len(df2.iloc[:,0])*percent_train_set)

        # Check for categorical variables
        cat_list = utils.find_cat(df2)

        if len(cat_list) > 0:
            print(f"Categorical variable found. Running Pandas dummy_variable encoding on {len(cat_list)} columns \n")
            df2 = utils.dummy_encode(df2, cat_list)
            print(f"Saving output dataset under {import_path}_NoCat.csv \n")
            df2.to_csv(f"{import_path}_NoCat.csv", index=False)
            self.dataloader = My_dataLoader(batchSize, f"{import_path}_NoCat.csv", n_test, "income", test_batch_size)
        
        else: # no categorical vars found
            self.dataloader = My_dataLoader(batchSize, import_path+'.csv', n_test, "income", test_batch_size)
        
        # Save data somewhere
#         1 - make dir inside experimetns with copy of params.yaml
#         2- dump json with other info in there
#         dataset_name = data['data_loading']['path']['value'].split('/')[-1]
        
#         with open(f'./experiments/{dataset_name}_{}.txt', 'w') as file:
#              file.write(json.dumps(exDict))
        

In [7]:
data_loader = PreProcessing("params.yaml")

Categorical variable found. Running Pandas dummy_variable encoding on: 
 [['Status_of_existing_checking_account', 'Credit_history', 'Purpose', 'Savings_account_bonds', 'Present_employment_since', 'Personal_status_and_sex', '_Other_debtors_or_guarantors', 'Property', 'Other_installment_plans', 'Housing', 'Job', 'foreign_worker']] 

Saving output dataset under ../GeneralDatasets/sanitizer_output/german_credit/n=1_alpha-gr-0.9_NoCat.csv 



In [10]:
# Box

class autoencoder(nn.Module):
    def __init__(self, in_dim, out_dim):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(in_dim, 128),
            nn.ReLU(True),
            nn.Linear(128, 64),
            nn.ReLU(True), nn.Linear(64, 12), nn.ReLU(True), nn.Linear(12, 3))
        self.decoder = nn.Sequential(
            nn.Linear(3, 12),
            nn.ReLU(True),
            nn.Linear(12, 64),
            nn.ReLU(True),
            nn.Linear(64, 128),
            nn.ReLU(True), nn.Linear(128, out_dim), nn.Tanh())

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

### Training

In [ ]:
wd = None
learning_rate = 0.001
num_epochs = 2
#######
in_dim = len(df2.iloc[0,:])
out_dim = len(df2.iloc[0,:])
#######

model = autoencoder(in_dim, out_dim).cuda()
train_loss = torch.nn.L1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=wd)


for epoch in range(num_epochs):
    for data in dataloader.train_data:
        img, _ = data
        img = img.view(img.size(0), -1)
        img = Variable(img).cuda()
        # Fprop
        output = model(img)
        loss = train_loss(output, img)
        # bprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        NEED TEST SET
        
    # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}'
          .format(epoch + 1, num_epochs, loss.data[0]))
    if epoch % 10 == 0:
        pic = to_img(output.cpu().data)
        save_image(pic, './mlp_img/image_{}.png'.format(epoch))

torch.save(model.state_dict(), './sim_autoencoder.pth')